In [22]:
from scipy.io import wavfile as wav
import matplotlib.pyplot as plt
from scipy.signal.windows import tukey
import numpy as np
import os as os

folder_path = r'C:\Users\pawma\Documents\Informatyka 5 semestr\Komunikacja czlowiek-komputer\Rozpoznawanie Plci\train'

score = 0
M_files = 0
K_files = 0
M = 0
K = 0

for filename in os.listdir(folder_path):
    
    if "M" in filename:
        M_files += 1
    if "K" in filename:
        K_files += 1

    file_path = os.path.join(folder_path, filename)

    sample_rate, data = wav.read(file_path)
    #print(data[0], data[100], data[1000])
    #print("czestotliwosc probkowania:", sample_rate, "Hz")
    #print("ilosc danych", len(data))


    mean_data = []
    mono = 1
    stereo = 2

    value_0 = 0
    for i in range(len(data)):
        if len(data.shape) == mono:
            if data[i] == 0:
                value_0 += 1
            mean_data.append(float(data[i]))
        elif len(data.shape) == stereo:
            if data[i][0] == 0 or data[i][1] == 0:
                value_0 += 1
            mean_data.append((float(data[i][0]) + float(data[i][1])) / 2)
        else:
            print("Nieznany typ danych")

    mean_data = [x for x in mean_data if x != 0]

    # Definiowanie parametrów okna tukeya
    window_length = len(data)  # Długość okna
    alpha = 0.4                # Parametr płaskości dla okna Tukey

    # Tworzenie okna tukeya
    tukey_window = tukey(window_length, alpha)


    multiplied_data = []
    for i in range(len(mean_data)):
        multiplied_data.append(mean_data[i] * tukey_window[i])


    T = len(multiplied_data)/sample_rate
    #print("czas:", T, "s")
    f = 1/T
    #print("czestotliwosc:", f, "Hz")
    #print("ilosc zer:", value_0)

    #fig = plt.figure(figsize=(10, 25), dpi=80) 

    #ax = fig.add_subplot(611)
    freqs = np.linspace(0, T, len(mean_data))
    #ax.plot(freqs, mean_data)
    #ax.plot(freqs, multiplied_data)


    #ax = fig.add_subplot(612)
    fft_spectrum = np.fft.fft(multiplied_data)
    fft_spectrum = np.abs(fft_spectrum)
    fft_spectrum = fft_spectrum[:len(fft_spectrum)//2]
    hps_spectrum = fft_spectrum.copy()

    freqs = np.linspace(0, sample_rate//2, len(fft_spectrum))
    #ax.plot(freqs, fft_spectrum)


    subplots = [613, 614, 615, 616]

    #Metoda HPS
    harmonics = 4  # =5
    for i in range(2, harmonics + 1):
        divided = fft_spectrum[::i]
        hps_spectrum[:len(divided)] *= divided


    min_freq = 60
    max_freq = 500

    # Wyzerowanie wartości poza zakresem min-max
    hps_spectrum[(freqs < min_freq) | (freqs > max_freq)] = 0

    # Znalezienie częstotliwości podstawowej
    fundamental_freq = np.argmax(hps_spectrum) * sample_rate / len(data)
    #print("Częstotliwość podstawowa:", round(fundamental_freq, 3), "Hz")

    # Narysowanie okna tukeya
    #fig = plt.figure(figsize=(7, 4), dpi=80)
    #ax = fig.add_subplot(111)
    #ax.plot(tukey_window, color = 'red') 


    if fundamental_freq < 172.5:
        if "M" in filename:
            score += 1
            M += 1
        else:
            print(filename, fundamental_freq)
    else:
        if "K" in filename:
            score += 1
            K += 1
        else:
            print(filename, fundamental_freq)
print(score)
print(M_files, K_files)
print()
print(M, M_files - M)
print(K_files - K, K)
    # Pliki do przetestowania:
    # 015_K.wav

C:\Users\pawma\AppData\Local\Temp\ipykernel_18884\3231070810.py:24: WavFileWarning: Reached EOF prematurely; finished at 613916 bytes, expected 613924 bytes from header.
  sample_rate, data = wav.read(file_path)


011_M.wav 185.09170355890302
018_K.wav 157.31132075471697
051_K.wav 155.21769971288634
054_K.wav 132.12410348360655
067_K.wav 99.81225296442688
068_K.wav 165.06728468899522
070_M.wav 238.3877840909091


C:\Users\pawma\AppData\Local\Temp\ipykernel_18884\3231070810.py:24: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, data = wav.read(file_path)


075_M.wav 176.53513941271552
081_K.wav 151.93330501274426
82
45 46

42 3
6 40
